In [104]:
import math
import json
import requests
import itertools
import numpy as np
import time
from datetime import datetime, timedelta
import pymongo
from pymongo import MongoClient

### Inicia Conexao com Mongo

In [105]:
db_client = MongoClient('localhost', 27017)
db_ic = db_client.ica
db_posts = db_ic.posts

In [106]:
def insert(id, timestamp):
    post = {
        "_id":id,
        "lancamento": timestamp
    }
    db_posts.insert_one(post)

def find(id):
    db_posts.find({"_id":id})

### Métodos Gerais

make request - monta o metodo que trata das tentativas do request

In [108]:
def make_request(uri, max_retries = 5):
    def fire_away(uri):
        response = requests.get(uri)
        assert response.status_code == 200
        return json.loads(response.content)    
    current_tries = 1
    while current_tries < max_retries:
        try:
            response = fire_away(uri)
            return response
        except:
            print(str(current_tries) + "...")
            time.sleep(0.5)
            current_tries += 1    
            return fire_away(uri)

---

get post between unit - Basicamente trabalha fazendo a coleta dos posts que tem o timestamp entre as datas fornecidas

Aqui lembrando que o unit insere 1 por vez no nosso banco de dados. Facilita o tratamento de insercao de chaves duplas, por isso tem preferencia

In [109]:
def get_post_between_unit(start_at,end_at):
    SIZE = 500
    URI_TEMPLATE = r'https://api.pushshift.io/reddit/search/submission?subreddit=depression&after={}&before={}&size={}'
    
    post_collection = make_request(URI_TEMPLATE.format(start_at, end_at, SIZE))['data']
    n = len(post_collection)
    
    
    print('adicionando primeira leva')
    
    for i in post_collection:
        try:
            db_posts.insert_one({'_id':i['id'], 'time':i['created_utc']})
        except pymongo.errors.DuplicateKeyError:
            print("ID Duplicado " + str(i['id']) + " pulando insercao")
        
        
    counter = 1
    while n == SIZE and start_at < end_at:
        counter +=1
        last = post_collection[-1]
        new_start_at = last['created_utc']
        print('adicionando nova leva, dia = ' + str(datetime.fromtimestamp(new_start_at)))
        
        post_collection = make_request(URI_TEMPLATE.format(new_start_at, end_at, SIZE))['data']
        n = len(post_collection)
        
        posts = []
        for i in post_collection:
            try:
                db_posts.insert_one({'_id':i['id'], 'time':i['created_utc']})
            except pymongo.errors.DuplicateKeyError:
                print("ID Duplicado " + str(i['id']) + " pulando insercao")


In [125]:
SIZE = 500
URI_TEMPLATE = r'https://api.pushshift.io/reddit/search/submission?subreddit=depression&size={}'
post_collection = make_request(URI_TEMPLATE.format(SIZE))['data']
n = len(post_collection)
    

In [128]:
post_collection[1]

{'all_awardings': [],
 'author': 'ghostlynihilist',
 'author_flair_css_class': None,
 'author_flair_richtext': [],
 'author_flair_text': None,
 'author_flair_type': 'text',
 'author_fullname': 't2_10gaecop',
 'author_patreon_flair': False,
 'can_mod_post': False,
 'contest_mode': False,
 'created_utc': 1562088449,
 'domain': 'self.depression',
 'full_link': 'https://www.reddit.com/r/depression/comments/c8c24t/do_you_ever_have_such_low_self_esteem_that_you/',
 'gildings': {},
 'id': 'c8c24t',
 'is_crosspostable': True,
 'is_meta': False,
 'is_original_content': False,
 'is_reddit_media_domain': False,
 'is_robot_indexable': True,
 'is_self': True,
 'is_video': False,
 'link_flair_background_color': '',
 'link_flair_richtext': [],
 'link_flair_text_color': 'dark',
 'link_flair_type': 'text',
 'locked': False,
 'media_only': False,
 'no_follow': False,
 'num_comments': 5,
 'num_crossposts': 0,
 'over_18': False,
 'parent_whitelist_status': 'no_ads',
 'permalink': '/r/depression/comments/c

Aqui podemos optar por inserir em grupos(bulk), o que torna mais rapida a insercao

In [122]:
%%time
inicio = math.floor((datetime.utcnow() - timedelta(days=1)).timestamp())
fim = math.ceil((datetime.utcnow() - timedelta(days=0)).timestamp())
get_post_between_unit(inicio, fim)

adicionando primeira leva
ID Duplicado c80loz pulando insercao
ID Duplicado c80m40 pulando insercao
ID Duplicado c80mub pulando insercao
ID Duplicado c80mz0 pulando insercao
ID Duplicado c80n8y pulando insercao
ID Duplicado c80ntt pulando insercao
ID Duplicado c80o6p pulando insercao
ID Duplicado c80p15 pulando insercao
ID Duplicado c80pfg pulando insercao
ID Duplicado c80ps1 pulando insercao
ID Duplicado c80r6j pulando insercao
ID Duplicado c80rcu pulando insercao
ID Duplicado c80rns pulando insercao
ID Duplicado c80sfa pulando insercao
ID Duplicado c80t9o pulando insercao
ID Duplicado c80tlv pulando insercao
ID Duplicado c80u44 pulando insercao
ID Duplicado c80ue3 pulando insercao
ID Duplicado c80uy5 pulando insercao
ID Duplicado c80v9g pulando insercao
ID Duplicado c80v9u pulando insercao
ID Duplicado c80wlm pulando insercao
ID Duplicado c80y6b pulando insercao
ID Duplicado c80yhb pulando insercao
ID Duplicado c80zv2 pulando insercao
ID Duplicado c811ds pulando insercao
ID Duplicado

ID Duplicado c8a331 pulando insercao
ID Duplicado c8a4bt pulando insercao
ID Duplicado c8a7d7 pulando insercao
ID Duplicado c8a8jh pulando insercao
ID Duplicado c8aa2u pulando insercao
ID Duplicado c8aa7y pulando insercao
ID Duplicado c8aanc pulando insercao
ID Duplicado c8abd2 pulando insercao
ID Duplicado c8ac6r pulando insercao
ID Duplicado c8ad4x pulando insercao
ID Duplicado c8adne pulando insercao
ID Duplicado c8adq4 pulando insercao
ID Duplicado c8adr1 pulando insercao
ID Duplicado c8ae7a pulando insercao
ID Duplicado c8af7w pulando insercao
ID Duplicado c8agc5 pulando insercao
ID Duplicado c8ah1n pulando insercao
ID Duplicado c8aha4 pulando insercao
ID Duplicado c8ai3k pulando insercao
ID Duplicado c8ajwu pulando insercao
ID Duplicado c8ajzj pulando insercao
ID Duplicado c8am1g pulando insercao
ID Duplicado c8anp3 pulando insercao
ID Duplicado c8anzi pulando insercao
ID Duplicado c8aoj9 pulando insercao
ID Duplicado c8aol8 pulando insercao
ID Duplicado c8aopb pulando insercao
I